In [1]:
%%capture
# Install the latest version of attk (if necessary)
#!pip install -U git+git://github.com/hipstas/audio-tagging-toolkit.git

In [2]:
import os
import attk
import csv
import joblib

import os
import numpy as np
import librosa
from IPython.display import display, Audio
import timeit
import random
from sklearn.externals import joblib

from sklearn.ensemble import ExtraTreesClassifier

from IPython.display import display, Audio

random.seed(1002) ## Seeding random number generator for reproducibility

In [ ]:
label_table=[]

with open("Potential_WGBH_tones_2887_1-sec_labels.csv") as csvfile:
    spamreader = csv.reader(csvfile)
    for row in spamreader:
        label_table.append(row)



In [ ]:
filenames=os.listdir("Potential_WGBH_tones_2887")
filenames[:5]

In [ ]:
!mkdir AAPB_test_tones
!mkdir AAPB_silence
!mkdir AAPB_bg_speakers
!mkdir AAPB_music

In [ ]:
out_dir="/home/sharedfolder/AAPB_test_tones/"

for row in label_table:
    filename=row[0]+'.wav'
    wav_path='Potential_WGBH_tones_2887/'+filename
    label=row[-1]
    if label == "Test tone":
        attk.subclip(wav_path,0,1,out_dir)

In [ ]:
out_dir="/home/sharedfolder/AAPB_silence/"

for row in label_table:
    filename=row[0]+'.wav'
    wav_path='Potential_WGBH_tones_2887/'+filename
    label=row[-1]
    if label == "Silence":
        attk.subclip(wav_path,0,1,out_dir)

In [ ]:
out_dir="/home/sharedfolder/AAPB_bg_speakers/"

for row in label_table:
    filename=row[0]+'.wav'
    wav_path='Potential_WGBH_tones_2887/'+filename
    label=row[-1]
    if label == "Background Speaker":
        attk.subclip(wav_path,0,1,out_dir)

In [ ]:
out_dir="/home/sharedfolder/AAPB_music/"

for row in label_table:
    filename=row[0]+'.wav'
    wav_path='Potential_WGBH_tones_2887/'+filename
    label=row[-1]
    if label == "Music":
        attk.subclip(wav_path,0,1,out_dir)

In [ ]:
dir_0 = 'AAPB_test_tones'
dir_1 = 'AAPB_silence'
dir_2 = 'AAPB_bg_speakers'
dir_3 = 'AAPB_music'

In [ ]:

mfccs_0 = []

for filename in os.listdir(dir_0):
    if '.wav' in filename:
        mfccs_0 += attk.get_mfccs_and_deltas(os.path.join(dir_0,filename))

mfccs_0[:1][0]

In [ ]:

mfccs_1 = []

for filename in os.listdir(dir_1):
    if '.wav' in filename:
        mfccs_1 += attk.get_mfccs_and_deltas(os.path.join(dir_1,filename))

mfccs_1[:1][0]

In [ ]:
mfccs_2 = []

for filename in os.listdir(dir_2):
    if '.wav' in filename:
        mfccs_2 += attk.get_mfccs_and_deltas(os.path.join(dir_2,filename))

mfccs_2[:1][0]

In [ ]:

mfccs_3 = []

for filename in os.listdir(dir_3):
    if '.wav' in filename:
        mfccs_3 += attk.get_mfccs_and_deltas(os.path.join(dir_3,filename))

mfccs_3[:1][0]

In [ ]:
# For 2-fold validation

X = mfccs_0[:len(mfccs_0)/2] + \
    mfccs_1[:len(mfccs_1)/2] + \
    mfccs_2[:len(mfccs_2)/2] + \
    mfccs_3[:len(mfccs_3)/2]

y = [0]*(len(mfccs_0)/2) + \
    [1]*(len(mfccs_1)/2) + \
    [1]*(len(mfccs_2)/2) + \
    [1]*(len(mfccs_3)/2)

print(len(X))
print(len(y))


In [ ]:
# For 2-fold validation

X_test = mfccs_0[len(mfccs_0)/2:] + \
    mfccs_1[len(mfccs_1)/2:] + \
    mfccs_2[len(mfccs_2)/2:] + \
    mfccs_3[len(mfccs_3)/2:]

y_test = [0]*len((mfccs_0[len(mfccs_0)/2:])) + \
    [1]*len((mfccs_1[len(mfccs_1)/2:])) + \
    [1]*len((mfccs_2[len(mfccs_2)/2:])) + \
    [1]*len((mfccs_3[len(mfccs_3)/2:]))

print(len(X_test))
print(len(y_test))

In [ ]:
# Using all data to train the actual classifier
## Skip this cell for 2-fold validation

X = mfccs_0 + mfccs_1 + mfccs_2 + mfccs_3

y = [0]*len(mfccs_0) + \
    [1]*len(mfccs_1) + \
    [1]*len(mfccs_2) + \
    [1]*len(mfccs_3)

print(len(X))
print(len(y))

In [ ]:
# Training a model and saving it to disk

from sklearn.ensemble import ExtraTreesClassifier

extra_trees = ExtraTreesClassifier().fit(X, y)

joblib.dump(extra_trees,'aapb_test_tones_extra_trees.pkl')
extra_trees=joblib.load('aapb_test_tones_extra_trees.pkl')

#extra_trees.score(X_test,y_test)

In [ ]:
!mkdir temp_clip

In [ ]:
# Ran the 2 cells below repeatedly to design my heuristic

In [ ]:
try:
    os.remove(clip_pathname)
except:
    pass

filename = random.choice(os.listdir('AAPB_Subset_4000'))
test_pathname = 'AAPB_Subset_4000/'+filename

print(filename)

attk.subclip(test_pathname,0,80,'temp_clip/')

clip_pathname='temp_clip/'+os.listdir('temp_clip/')[0]

test_mfccs = attk.get_mfccs_and_deltas(clip_pathname)

print(clip_pathname)

results = extra_trees.predict(test_mfccs)  ## Predicting new observation

print(list(results[:150])+['...'])
print(np.mean(attk.smooth(results)[:50]))

if np.mean(attk.smooth(results)[:50])<0.6:
    display(Audio(clip_pathname))

In [ ]:
# Second positions where test tones begin and end:

confirmed_tones=[]

if True: #np.mean(attk.smooth(results)[:60])<0.5:
    smoothed_ranges = attk.labels_to_ranges([round(item) \
                                for item in attk.smooth(results,window_len=10)],label=0)
    if len(smoothed_ranges)>0:
        for pair in smoothed_ranges:
            start, end = pair
            if (end-start) > 5:
                start_time = (80.0/len(results))*start
                end_time = (80.0/len(results))*(end+5)
                confirmed_tones.append((start_time,end_time))
    else:
        print("No tone.")

print(confirmed_tones)

In [ ]:
######## RESUME HERE WHEN USING PRE-TRAINED MODEL ########

from sklearn.ensemble import ExtraTreesClassifier

extra_trees=joblib.load('aapb_test_tones_extra_trees.pkl')

##########################################################

NameError: name 'joblib' is not defined

In [ ]:
## Assembling the batch run

In [ ]:
## Creating a list of all media pathnames within a directory (including subdirectories)

media_paths = attk.find_media_paths('AAPB_Subset_4000/')

media_paths[:3]+['...']

In [6]:
## Creating a list of files that have already been classified (in case we get interrupted)

completed_filenames=[]

with open('Potential_WGBH_tones_2887_classifier_4K_batch.csv') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        try:
            completed_filenames.append(row[0])
        except:
            print("Problem loading: "+str(row))

In [7]:
try: os.mkdir('temp_clip')
except: pass

In [ ]:
import random
import csv

import timeit

errors=[]

for media_path in media_paths: 
#for media_path in random.sample(media_paths,10):  ## Alternative for classifying 
                                                   ## a random subset
    print(media_path)

    ## Starting timer
    tic=timeit.default_timer()
    
    try:
        if media_path.split('/')[-1] not in completed_filenames: ## Checking whether file
                                                                 ## been classified already

            ## Creating temporary WAV excerpt and extracting features
            clip_pathname = attk.subclip(media_path,0,80,'temp_clip/')  ## first 80 seconds
            test_mfccs = attk.get_mfccs_and_deltas(clip_pathname)
            
            ## Classifying audio
            results = extra_trees.predict(test_mfccs)
            print(list(results[:150])+['...'])
            
            ## Smoothing and filtering output
            smoothed_ranges = attk.labels_to_ranges( \
                        [round(item) for item in attk.smooth(results, window_len=10)], \
                        label=0)
            
            confirmed_tones=[]
            
            if len(smoothed_ranges)>0:
                for pair in smoothed_ranges:
                    start, end = pair
                    if (end-start) > 5:
                        start_time = (80.0/len(results))*start         
                        end_time = (80.0/len(results))*(end+5)
                        confirmed_tones.append((start_time,end_time))
            
            ## Writing positive classification ranges to CSV
            csvfile = open('Potential_WGBH_tones_2887_classifier_4K_batch.csv','a')
            csvwriter = csv.writer(csvfile)
            
            for pair in confirmed_tones:
                start, end = pair
                csvwriter.writerow([media_path.split('/')[-1],str(round(start,4)),str(round(end,4))])
            
            if len(confirmed_tones)==0:
                csvwriter.writerow([media_path.split('/')[-1],'',''])
            
            csvfile.close()
            
            ## Deleting temporary WAV clip
            try: os.remove(clip_pathname)
            except: print "** Error: Can't remove "+clip_pathname

    ## Printing and storing errors
    except Exception as e:
        error_pair=(media_path,e)
        print(error_pair)
        errors.append(error_pair)
    
    ## Printing time elapsed
    print("Time elapsed: "+str(timeit.default_timer() - tic))


for item in errors:
    print str(item)+'\n'

## *Multiprocessing*

In [8]:
## Function version of the script above, for multithreading

def classify_and_save(media_path):
    
    ## Starting timer
    tic=timeit.default_timer()
    
    try:
        if media_path.split('/')[-1] not in completed_filenames: ## Checking whether file
                                                                 ## been classified already

            ## Creating temporary WAV excerpt and extracting features
            clip_pathname = attk.subclip(media_path,0,80,'temp_clip/')  ## first 80 seconds
            test_mfccs = attk.get_mfccs_and_deltas(clip_pathname)
            
            ## Classifying audio
            results = extra_trees.predict(test_mfccs)
            print(list(results[:150])+['...'])
            
            ## Smoothing and filtering output
            smoothed_ranges = attk.labels_to_ranges( \
                        [round(item) for item in attk.smooth(results, window_len=10)], \
                        label=0)
            
            confirmed_tones=[]
            
            if len(smoothed_ranges)>0:
                for pair in smoothed_ranges:
                    start, end = pair
                    if (end-start) > 5:
                        start_time = (80.0/len(results))*start         
                        end_time = (80.0/len(results))*(end+5)
                        confirmed_tones.append((start_time,end_time))
            
            ## Writing positive classification ranges to CSV
            with open('Potential_WGBH_tones_2887_classifier_4K_batch.csv','a') as csvfile:
                csvwriter = csv.writer(csvfile)
            
                for pair in confirmed_tones:
                    start, end = pair
                    csvwriter.writerow([media_path.split('/')[-1],str(round(start,4)),str(round(end,4))])
            
                if len(confirmed_tones)==0:
                    csvwriter.writerow([media_path.split('/')[-1],'',''])
            
            ## Deleting temporary WAV clip
            try: os.remove(clip_pathname)
            except: print "** Error: Can't remove "+clip_pathname

    ## Printing and storing errors
    except Exception as e:
        error_pair=(media_path,e)
        print(error_pair)
        errors.append(error_pair)
    
    time.sleep(random.random())
    
    ## Printing time elapsed
    print("Time elapsed: "+str(timeit.default_timer() - tic))
    if float(timeit.default_timer() - tic) > 0.5:
        print "***"



In [9]:
import random
import csv
import time
import timeit

errors=[]





In [10]:

def update_completed_list():
    completed_filenames=[]

    with open('Potential_WGBH_tones_2887_classifier_4K_batch.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            try:
                completed_filenames.append(row[0])
            except:
                print("Problem loading: "+str(row))

update_completed_list()
print len(list(set(completed_filenames)))

4669


In [ ]:
%%capture

# Classifying in parallel batches of 2

import time

from multiprocessing import Pool



for i in range(1000):
    tic=timeit.default_timer()
    pool_media_paths=[]
    
    pool = Pool(2)
    
    for j in range(2):
        media_path = random.choice(media_paths)
        update_completed_list()
        counter=0
        while media_path.split('/')[-1] in completed_filenames:
            media_path = random.choice(media_paths)
            counter+=1
            if counter>10000:
                print("Looks like you might be done.")
                break
                break
                break
        pool_media_paths.append(media_path)

    print(pool_media_paths)
    pool.map(classify_and_save, pool_media_paths)
    print("Time elapsed: "+str(timeit.default_timer() - tic))



In [ ]:
%%capture

# Classifying in multithreaded batches of 4

import time

from multiprocessing import Process

tic=timeit.default_timer()

for i in range(100):
    for j in range(4):
        media_path = random.choice(media_paths)
        update_completed_list()
        counter=0
        while media_path.split('/')[-1] in completed_filenames:
            media_path = random.choice(media_paths)
            counter+=1
            if counter>10000:
                print("Looks like you might be done.")
                break
                break
                break
        print(media_path)
        p = Process(target=classify_and_save, args=(media_path,))
        p.start()
        time.sleep(2.7)
        print("*****")
    p.join()

In [ ]:
print("Time elapsed: "+str(timeit.default_timer() - tic))

In [ ]:
print(errors)